In [1]:
import pandas as pd
from data.constants import DNC_START, DNC_END

In [2]:
# Exported from:
# https://www.baseball-reference.com/teams/CHC/2024-schedule-scores.shtml
# https://www.baseball-reference.com/teams/CHW/2024-schedule-scores.shtml
# https://www.pro-football-reference.com/teams/chi/2024.htm#all_games
# https://www.pro-football-reference.com/years/2024/attendance.htm
# https://www.basketball-reference.com/teams/CHI/2025_games.html
# https://www.basketball-reference.com/teams/CHI/2024_games.html
# https://www.hockey-reference.com/teams/CHI/2025_games.html
# https://www.hockey-reference.com/teams/CHI/2024_games.html

# I manually named the home/away column in most of these, 
# to avoid future column order errors integrating other years.

# Inputs

In [3]:
cubs_file = "../data/raw/scores-2024-cubs.csv"
sox_file = "../data/raw/scores-2024-whitesox.csv"
hawks24_file = "../data/raw/scores-2024-blackhawks.csv"
hawks25_file = "../data/raw/scores-2025-blackhawks.csv"
bulls24_file = "../data/raw/scores-2024-bulls.csv"
bulls25_file = "../data/raw/scores-2025-bulls.csv"
stars_file = "../data/raw/scores-2024-redstars.csv"
nfl_file = "../data/raw/scores-2024-nfl.csv"
bears_file = "../data/raw/scores-2024-bears.csv"
fire_file = "../data/raw/scores-2024-fire.csv"
attend_out = '../data/interim/sports.csv'

In [4]:
cubs = pd.read_csv(cubs_file)
sox = pd.read_csv(sox_file)
hawks = pd.concat([pd.read_csv(hawks24_file),pd.read_csv(hawks25_file)])
bulls = pd.concat([pd.read_csv(bulls24_file),pd.read_csv(bulls25_file)])
stars = pd.read_csv(stars_file, sep='\t', thousands=",")
fire = pd.read_csv(fire_file, sep='\t', thousands=",")
nfl = pd.read_csv(nfl_file)
bears = pd.read_csv(bears_file, header=[0,1])

# Clean

In [5]:
def clean_baseball(df):
    months = df['Date'].str.extract(r"[a-zA-Z]+ ([a-zA-Z]+) [0-9]+", expand=False)
    days = df['Date'].str.extract(r"[a-zA-Z]+ [a-zA-Z]+ ([0-9]+)", expand=False)
    dates = pd.Series([f"2024-{m}-{d.rjust(2,'0')}" for m,d in zip(months, days)], name='date')
    dates = pd.to_datetime(dates, format="%Y-%b-%d")

    home = (df['@'] != '@').rename('home')
    attend = df['Attendance'].rename('attendance')

    df = pd.concat([dates, home, attend], axis=1)
    return df

In [6]:
cubs_clean = clean_baseball(cubs)

In [7]:
sox_clean = clean_baseball(sox)

In [8]:
dates = pd.to_datetime(hawks['Date'].rename('date'), format="%Y-%m-%d")
home = (hawks['@'] != '@').rename('home')
attend = hawks['Att.'].rename('attendance')

In [9]:
hawks_clean = pd.concat([dates, home, attend], axis=1)
hawks_clean = hawks_clean.loc[hawks_clean.attendance.notna()] # Drops games that haven't been played yet.

In [10]:
dates = pd.to_datetime(bulls['Date'].rename('date'), format="%a %b %d %Y")
home = (bulls['@'] != '@').rename('home')
attend = bulls['Attend.'].rename('attendance')

bulls_clean = pd.concat([dates, home, attend], axis=1)
bulls_clean = bulls_clean.loc[bulls_clean.attendance.notna()] # Drops games that haven't been played yet.

In [11]:
def clean_soccer(df):
    df.columns = [x.strip(' ') for x in df.columns]
    dates = pd.to_datetime(df['Date'].rename('date'), format="%Y-%m-%d")
    home = (df['Venue'] == 'Home').rename('home')
    attend = df['Attendance'].rename('attendance')

    return pd.concat([dates, home, attend], axis=1)

In [12]:
stars_clean = clean_soccer(stars)

In [13]:
fire_clean = clean_soccer(fire)

In [14]:
cols = [('' if 'Unnamed' in i else i) + ('' if 'Unnamed' in j else j) for i,j in bears.columns.values]
bears.columns = ['Col'+str(i) if c == '' else c for i,c in enumerate(cols)]

attend = nfl.loc[nfl['Tm'] == 'Chicago Bears'].filter(regex=r'^Week \d+$').T
attend = attend.iloc[:, 0].rename('attendance')
attend = pd.to_numeric(attend.map(lambda x: pd.NA if x == 'Bye' else x))
attend = attend.reset_index()


weeks = 'Week ' + bears['Week'].astype(str).rename('index')
home = (bears['Col8'] != '@').rename('home')
months = bears['Date'].str.extract(r"([a-zA-Z]+) [0-9]+", expand=False)
days = bears['Date'].str.extract(r"[a-zA-Z]+ ([0-9]+)", expand=False)
dates = pd.Series([f"2024-{m}-{d}" for m,d in zip(months, days)], name='date')
dates = pd.to_datetime(dates, format="%Y-%B-%d", errors='coerce')

bears_clean = pd.concat([weeks,home,dates],axis=1).merge(attend, on='index').drop(columns=['index'])

# Merge

In [15]:
attend = pd.concat([
    cubs_clean,
    sox_clean,
    hawks_clean,
    bulls_clean,
    stars_clean,
    fire_clean,
    bears_clean],
    names=['team','index'],
    keys=['cubs','whitesox',
          'blackhawks','bulls',
          'redstars','fire','bears']) \
    .reset_index('team')

In [16]:
stadiums = {'cubs':'wrigley', 'whitesox':'guaranteed rate', 
            'blackhawks': 'united center', 'bulls': 'united center',
            'redstars': 'seatgeek', 'fire': 'soldier', 'bears': 'soldier'}
attend['stadium'] = attend['team'].map(stadiums)

In [17]:
attend = attend.loc[attend['home']].drop(columns=['home'])

In [18]:
print("Dropping these NaN attendance games:")
print(attend[attend.attendance.isna() | (attend.attendance == 0)][['team','date']])
attend = attend.loc[attend.attendance.notna()]

Dropping these NaN attendance games:
           team       date
index                     
16     whitesox 2024-04-17
41     whitesox 2024-05-14
93     whitesox 2024-07-10
132    whitesox 2024-08-27
6         bears        NaT


# Add DNC Data

In [19]:
dnc = [
    {'date': d, 'team': 'DNC', 'attendance': 1.5e4, 'stadium':'united center'}
    for d in pd.date_range(DNC_START, DNC_END)
] + [
    {'date': d, 'team': 'DNC', 'attendance': 1.5e4, 'stadium':'mccormick place'}
    for d in pd.date_range(DNC_START, DNC_END)
]
dnc = pd.DataFrame.from_records(dnc)

In [20]:
attend = pd.concat([attend, dnc], ignore_index=True)

# Exports

In [21]:
attend.to_csv(attend_out, index=False)